# **Measuring Bias in classification**


This notebook is a tutorial on auditing bias within a binary classification task. We will use the holisticai library both in the **data exploration** and **measure bias** sections, introducing some of the functions we have created to help study algorithmic bias.

The sections are organised as follows :
1. Load the data : we load the law school dataset as a pandas DataFrame
2. Data Exploration : some preliminary analysis of the data
3. Train a Model : we train a simple logistic regression model (sklearn)
4. Measure Bias : we compute a few bias metrics, and comment on their meaning

## **Load the data**

First of all, we need to import the required packages to perform our bias analysis and mitigation. You will need to have the `holisticai` package installed on your system, remember that you can install it by running: 
```bash
!pip install holisticai[all]
```

In [1]:
# Imports
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

We host a few example datasets on the holisticai library for quick loading and experimentation. Here we load and use the Law School dataset. The goal of this dataset is the prediction of the binary attribute 'bar' (whether a student passes the law school bar). The protected attributes are race and gender. We pay special attention to race in this case, because preliminary exploration hints there is strong inequality on that sensitive attribute.

In [2]:
import os,sys

dirpath = os.path.join( '/'.join(os.getcwd().split("/")[:-6]), 'src')
print(dirpath)
sys.path.insert(0, dirpath)

/home/cristian/github/holisticai/src


In [3]:
from holisticai.datasets import load_dataset

dataset = load_dataset('law_school')
dataset

Dataset({
        features: ['x', 'y', 'p_attr'],
        num_rows: 20800
    })

## **Data Exploration**

We import some of the holisticai plotters for quick exploration of the data.

In [22]:
%%capture params

frequency = 1
amplitude = 1

In [4]:
from holisticai.plots.bias import bias_data_exploration

bias_data_exploration(learning_task="binary_classification", dataset=dataset)

The proportion of white people in law school is very high, allready we observe there is a big representation issue.

We also observe that the white group has a much higher pass rate (within the dataset) than the non-white group.

## **Train a model**


Here we train a Logistic Regression classifier.

In [5]:
dataset = dataset.train_test_split(test_size=0.3)
dataset

DatasetDict({
    train: Dataset({
        features: ['x', 'y', 'p_attr'],
        num_rows: 14560
    }),
    test: Dataset({
        features: ['x', 'y', 'p_attr'],
        num_rows: 6240
    })
})

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# train a model, do not forget to standard scale data
train = dataset['test']
test = dataset['test']

scaler = StandardScaler()
X_train_t = scaler.fit_transform(train['x'])
LR = LogisticRegression(random_state=42, max_iter=500)
LR.fit(X_train_t, train['y'])
X_test_t = scaler.transform(test['x'])
y_pred = LR.predict(X_test_t)

In [7]:
#from holisticai.metrics.efficacy import classification_efficacy_metrics
#classification_efficacy_metrics(y_pred, y_test)

## **Measure bias**

The holisticai.metrics.bias module contains a range of metrics useful in evaluating the fairness of algorithmic decisions. In this case we use only a few of the metrics relevant to a classification task.

In [8]:
# import some bias metrics
from holisticai.metrics.bias import statistical_parity
from holisticai.metrics.bias import disparate_impact
from holisticai.metrics.bias import accuracy_diff

In [9]:
# set up groups, prediction array and true (aka target/label) array.
y_pred  = LR.predict(X_test_t)          # prediction vector
y_true  = test['y']                        # true vector

In [10]:
group_a = test['p_attr']['group_a']
group_b = test['p_attr']['group_b']

from holisticai.plots.bias import bias_model_exploration

bias_model_exploration("binary_classification", test, y_pred)

In [11]:
# compute statistical parity

statistical_parity(group_a, group_b, y_pred)

0.13301034401147172

The statistical parity indicates the difference in success rate between non-white and white groups. In this case it is outside of ranges considered fair (-0.1, 0.1).

In [12]:
# compute disparate impact
disparate_impact(group_a, group_b, y_pred)

1.1553789007357487

The disparate impact on the other hand is within the fair range (0.8, 1.2). This shows the importance of considering many different metrics to get a holistic picture of the situation.

In [13]:
# compute accuracy difference
accuracy_diff(group_a, group_b, y_pred, y_true)

0.1322084394882289

The above metric is different from the first two in that it also uses the target values, this is an equality of opportunity metric. A value of -0.12 shows that the classifier we trained is less accurate on non-white group than white group. This is expected because of the data imbalance.

The above shows the same result as accuracy_diff metric in plot form.

**Equality of outcome metrics (batch computation)**

In [14]:
# import function for batch computation
from holisticai.metrics.bias import classification_bias_metrics

In [15]:
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='equal_outcome')

,Value,Reference
Metric,,
Statistical Parity,0.133010,0
Disparate Impact,1.155379,1
Four Fifths Rule,0.865517,1
Cohen D,0.774900,0
2SD Rule,21.873687,0


**Equality of opportunity metrics (batch computation)**

In [16]:
classification_bias_metrics(group_a, group_b, y_pred, y_true, metric_type='equal_opportunity')

,Value,Reference
Metric,,
Equality of Opportunity Difference,0.057874,0
False Positive Rate Difference,0.301488,0
Average Odds Difference,0.179681,0
Accuracy Difference,0.132208,0


For instance the false positive rate difference of 0.34 hints that white people are more likely to be missclassified as passing the bar than non-whites.

We can show all group bias metrics by setting 'metric_types' as 'group':

In [17]:
classification_bias_metrics(group_a=group_a, group_b=group_b, y_pred=y_pred, y_true=y_true, metric_type='group')

,Value,Reference
Metric,,
Statistical Parity,0.133010,0
Disparate Impact,1.155379,1
Four Fifths Rule,0.865517,1
Cohen D,0.774900,0
2SD Rule,21.873687,0
Equality of Opportunity Difference,0.057874,0
False Positive Rate Difference,0.301488,0
Average Odds Difference,0.179681,0
Accuracy Difference,0.132208,0


We can show all individual bias metrics by setting 'metric_types' as 'individual'.

In [18]:
classification_bias_metrics(y_pred=y_pred, y_true=y_true, X=X_test_t, metric_type='individual')

,Value,Reference
Metric,,
Theil Index,0.038864,0
Generalized Entropy Index,0.039127,0
Coefficient of Variation,0.395609,0
Consistency Score,0.980801,1


You can pass specific parameters for individual fairness metrics:

In [19]:
individual_kargs = {
    'generalized_entropy_index__alpha': 2,
    'consistency_score__n_neighbors' : 5
}
classification_bias_metrics(y_pred=y_pred, y_true=y_true, X=X_test_t, metric_type='individual', **individual_kargs)

,Value,Reference
Metric,,
Theil Index,0.038864,0
Generalized Entropy Index,0.039127,0
Coefficient of Variation,0.395609,0
Consistency Score,0.980801,1
